In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import os
import jsonlines
import ujson
import pandas as pd
from tqdm import tqdm
import numpy as np
import random
from random import randrange, sample
from collections import Counter, defaultdict, OrderedDict
from itertools import chain, combinations
from operator import itemgetter
import scipy
import sys
import argparse

import wilds
import configs
from wilds.get_dataset import get_dataset

# LOAD AND INSPECT SELF TRAINING ITERATIONS

In [27]:
self_train_iterations = {
    # civil comments runs
    'civilcomments_0.7thresh':{
        'iter_files': ['subsample_civilcomments.csv', 'iter1.csv', 'iter2.csv', 'iter3.csv'],
        'path': f"/dfs/scratch1/simran/cs329d/CS329D_SelfTraining/wilds/data/civilcomments_v1.0/baseline0.7thresh/",
        'prediction_path':'/dfs/scratch1/simran/cs329d/CS329D_SelfTraining/wilds/selftrain_test_civil/round2/baseline0.7thresh/',
        'dataset':'civilcomments'
    },
    'civilcomments_0.8thresh':{
        'iter_files': ['subsample_civilcomments.csv', 'iter1.csv', 'iter2.csv', 'iter3.csv'],
        'path': f"/dfs/scratch1/simran/cs329d/CS329D_SelfTraining/wilds/data/civilcomments_v1.0/",
        'prediction_path':'/dfs/scratch1/simran/cs329d/CS329D_SelfTraining/wilds/selftrain_test_civil/',
        'dataset':'civilcomments'
    },
    'civilcomments_0.9thresh':{
        'iter_files': ['subsample_civilcomments.csv', 'iter1.csv', 'iter2.csv', 'iter3.csv'],
        'path': f"/dfs/scratch1/simran/cs329d/CS329D_SelfTraining/wilds/data/civilcomments_v1.0/baseline0.9thresh/",
        'prediction_path':'/dfs/scratch1/simran/cs329d/CS329D_SelfTraining/wilds/selftrain_test_civil/round2/baseline0.9thresh/',
        'dataset':'civilcomments'
    },
    'civilcomments_0.95thresh':{
        'iter_files': ['subsample_civilcomments.csv', 'iter1.csv', 'iter2.csv', 'iter3.csv'],
        'path': f"/dfs/scratch1/simran/cs329d/CS329D_SelfTraining/wilds/data/civilcomments_v1.0/baseline0.95thresh/",
        'prediction_path':'/dfs/scratch1/simran/cs329d/CS329D_SelfTraining/wilds/selftrain_test_civil/round2/baseline0.95thresh/',
        'dataset':'civilcomments'
    },
    
    # amazon runs
    'amazon_0.8thresh':{
        'iter_files': ['subsample_amazon.csv', 'iter1.csv', 'iter2.csv', 'iter3.csv'],
        'path': f"/dfs/scratch1/simran/cs329d/CS329D_SelfTraining/wilds/data/amazon_v2.0/baseline0.8thresh/",
        'prediction_path':'/dfs/scratch1/simran/cs329d/CS329D_SelfTraining/wilds/selftrain_test_amazon/round1/baseline0.8thresh/',
        'dataset':'amazon'
    },
    'amazon_0.9thresh':{
        'iter_files': ['subsample_amazon.csv', 'iter1.csv', 'iter2.csv', 'iter3.csv'],
        'path': f"/dfs/scratch1/simran/cs329d/CS329D_SelfTraining/wilds/data/amazon_v2.0/baseline0.9thresh/",
        'prediction_path':'/dfs/scratch1/simran/cs329d/CS329D_SelfTraining/wilds/selftrain_test_amazon/round1/baseline0.9thresh/',
        'dataset':'amazon'
    },
    'amazon_0.9thresh':{
        'iter_files': ['subsample_amazon.csv', 'iter1.csv', 'iter2.csv', 'iter3.csv'],
        'path': f"/dfs/scratch1/simran/cs329d/CS329D_SelfTraining/wilds/data/amazon_v2.0/baseline0.9thresh/",
        'prediction_path':'/dfs/scratch1/simran/cs329d/CS329D_SelfTraining/wilds/selftrain_test_amazon/round1/baseline0.9thresh/',
        'dataset':'amazon'
    }
}

run_key = 'amazon_0.9thresh'

In [28]:
datasets = []
for it in self_train_iterations[run_key]['iter_files']:
    file_path = f"{self_train_iterations[run_key]['path']}{it}"
    if self_train_iterations[run_key]['dataset'] == 'civilcomments':
        metadata_df = pd.read_csv(file_path,index_col='id')
    else:
        metadata_df = pd.read_csv(file_path)
    datasets.append(metadata_df)
    

In [29]:
for dataset in datasets:
    train_split = dataset[dataset['split'] == 0]
    dev_split = dataset[dataset['split'] == 1]
    test_split = dataset[dataset['split'] == 2]
    print(f"Full dataset: {len(dataset)}.")
    print(f"Train split: {len(train_split)}.")
    print(f"Dev split: {len(dev_split)}.")
    print(f"Test split: {len(test_split)}.\n")

Full dataset: 150000.
Train split: 50000.
Dev split: 25000.
Test split: 25000.

Full dataset: 150000.
Train split: 65243.
Dev split: 25000.
Test split: 25000.

Full dataset: 150000.
Train split: 66230.
Dev split: 25000.
Test split: 25000.

Full dataset: 150000.
Train split: 66491.
Dev split: 25000.
Test split: 25000.



## Between iterations, which types of unlabeled examples were labeled most often?
- based on gold label
- based on group
- based on observations


In [145]:
sub_df.sample(1)

,reviewerID,asin,overall,reviewTime,unixReviewTime,reviewText,summary,verified,category,reviewYear,split,y,id
1940,A2YQ0ULTPTMHJS,0007181582,5.0,"12 22, 2016",1482364800,Carson and his partner are part of a new inves...,Great first in a series,False,Books,2016,0,4,1940


In [159]:
# extract differences 
sub_df = datasets[0][datasets[0]['split'] == 0]
original_train = list(sub_df.index)

sub_df = datasets[1][datasets[1]['split'] == 0]
iter1_train = list(sub_df.index)
new_train_idx = [idx for idx in iter1_train if idx not in original_train]

In [147]:
datasets[0].sample(1)

,id,comment_text,split,created_date,publication_id,parent_id,article_id,rating,funny,wow,...,asian_latino_etc,disability_any,identity_any,num_identities,more_than_one_identity,na_gender,na_orientation,na_religion,na_race,na_disability
82750,5486585,Er..you need 51 Senators to reconcile..the bil...,1,2017-06-27 18:43:16.174624+00,102,5486144.0,348910,approved,0,0,...,0,0,0,0.0,False,1,1,1,1,1


In [168]:
def get_toxicity(row):
    try:
        toxicity = row['toxicity']
        if toxicity < 0.5:
            toxicity = 0
        else:
            toxicity = 1
    except:
        toxicity = row['toxicity'].values
        if toxicity < 0.5:
            toxicity = 0
        else:
            toxicity = 1
    return toxicity

train_label_proportions = Counter()
train_group_proportions = Counter()
test_label_proportions = Counter()
test_group_proportions = Counter()
for ind, row in tqdm(datasets[0].iterrows()):
    if row['split'] == 0:
        if self_train_iterations[run_key]['dataset'] == 'civilcomments':
            label = get_toxicity(row)
            for var in identity_vars:
                var_val = row[var]
            train_group_proportions[var] += var_val
        elif self_train_iterations[run_key]['dataset'] == 'amazon':
            label = row['y']
            train_group_proportions[row['reviewerID']] += 1
        train_label_proportions[label] += 1
        
    if row['split'] == 3:
        if self_train_iterations[run_key]['dataset'] == 'civilcomments':
            label = get_toxicity(row)
            for var in identity_vars:
                var_val = row[var]
            test_group_proportions[var] += var_val
        elif self_train_iterations[run_key]['dataset'] == 'amazon':
            label = row['y']
            test_group_proportions[row['reviewerID']] += 1
        test_label_proportions[label] += 1
        

145180it [00:18, 8000.26it/s]


In [ ]:
# based on gold label
identity_vars = [
            'male',
            'female',
            'LGBTQ',
            'christian',
            'muslim',
            'other_religions',
            'black',
            'white']

gold_label_counter = Counter()
group_counter = Counter()
for idx in tqdm(new_train_idx):
    row = datasets[0][datasets[0].index == idx]
    if self_train_iterations[run_key]['dataset'] == 'civilcomments':
        label = get_toxicity(row)
        var_str= ''
        for var in identity_vars:
            var_val = row[var].values[0]
            if var_val != 0:
                if var_str:
                    print(f"Was {var_str}, now {var}")
                var_str = var
                
        group_counter[var] += var_val
        
    elif self_train_iterations[run_key]['dataset'] == 'amazon':
        label = row['y'].values[0]
        group_counter[row['reviewerID'].values[0]] += 1
    print()
    gold_label_counter[label] += 1
    

In [119]:
print(test_label_proportions)
print(train_label_proportions)     
print(gold_label_counter)

Counter({4: 28405, 3: 14144, 2: 5031, 1: 1608, 0: 812})
Counter({4: 28727, 3: 14820, 2: 4582, 1: 1343, 0: 528})
Counter({4: 13221, 3: 1928, 2: 75, 0: 12, 1: 7})


In [120]:
print("TRAIN / TEST / ADDED LABELS")
for group in test_group_proportions.keys():
    print(f"{group}, {train_group_proportions[group]}, {test_group_proportions[group]}, {group_counter[group]}")

TRAIN / TEST / ADDED LABELS
A1LVKURA7X0WJ2, 0, 38, 10
A1S41XZ36HCUCP, 0, 41, 8
A2JOW1PW9GRCA3, 0, 38, 4
A3G4NWH4ICYBQQ, 0, 35, 29
A2W4014S06NAIT, 0, 32, 4
A1OLR1XE9COZUT, 0, 32, 28
A1I6BG5XRGOUJJ, 0, 41, 3
A3JTJ1E8NK7DH1, 0, 40, 12
A3LGT6UZL99IW1, 0, 40, 6
A1B4I7MHW9DX21, 0, 38, 7
AVNS35OHR3D76, 0, 37, 3
A2ESKW2RO8BOPU, 0, 35, 11
A3W3XC01BFJJMR, 0, 34, 18
A24O617GI0GZIJ, 0, 39, 18
A2II09GQGWOMTQ, 0, 39, 12
A3FVAWZNKW9GX, 0, 41, 14
A9IK5K9R5VCAN, 0, 41, 1
A19YOD91OWKTB7, 0, 39, 26
A2YATE2EMMY3N5, 0, 31, 5
AGN1CZ1ETR6DN, 0, 33, 2
AMUP8DYE7EAN2, 0, 33, 1
A1NMUPG3KMWRZM, 0, 45, 12
A2GU5SHR2DC29H, 0, 37, 20
A2L345QUOAS3UT, 0, 39, 15
AVOPHZPWGEODY, 0, 41, 24
A2ASKRCB2V7O56, 0, 41, 5
A3VIOCJZ22JZXT, 0, 40, 17
A3HQAPXAJV7PRX, 0, 45, 12
A31GICBOUJWVUE, 0, 34, 4
A19GJX6TTSK0C0, 0, 33, 6
AT88ONKPM9W9R, 0, 40, 14
AO74N9DWE1X0J, 0, 42, 12
A1JJ6IQN8OQDGQ, 0, 42, 15
A2G3RB76XWMP53, 0, 40, 6
A84EXNO674TBX, 0, 37, 21
ANXR2HLQMFWGG, 0, 36, 36
A3CKIH7X8QDIQT, 0, 47, 17
A36PA4XPATJSKX, 0, 37, 3
A3CS4U2S7C

AYT4FJYVCHYLE, 0, 42, 6
A1GD1G8XXPRYQ8, 0, 34, 5
A3GFQDC7240EZR, 0, 33, 9
A35BKLDD2E04E6, 0, 36, 12
A1YEDUONEOPVSV, 0, 37, 22
A7KQB3WGH6HT7, 0, 37, 9
AHLVLW2TZX1LA, 0, 40, 14
A2VH0UT5EQFB6P, 0, 39, 8
AS28WVBBZ3G99, 0, 37, 15
A1FUH1O6FCTUYG, 0, 32, 24
A2V5E88FME47TG, 0, 34, 9
APRP8UBWTQ15J, 0, 40, 18
A2VHGFUCYDJ09G, 0, 41, 0
A23TQDWMUA4F6Y, 0, 41, 21
A23IUL4X7JNQ47, 0, 41, 28
A15WWOTR3FBTD0, 0, 30, 24
A3K1A3MUYKQ9B9, 0, 40, 8
AFLUSKCVBJ8UG, 0, 44, 11
A2V1ZGT9AE5FC6, 0, 37, 25
A385T3OAKM0N5H, 0, 30, 19
A3N51LNLFRBVNY, 0, 40, 25
A2BU601JNTT5WS, 0, 43, 15
A3LNJX5X6ZKYNW, 0, 36, 21
ASTQ00L6ORVBS, 0, 37, 19
A3TCEX54TCIEES, 0, 41, 13
A3K37QVJCLI6AQ, 0, 40, 20
A3GQY7UCN7FAWJ, 0, 38, 14
A1FTB3KM7X78OX, 0, 42, 0
A3KI93AT8FAXI1, 0, 44, 12
AZD488SA9QMYF, 0, 40, 22
A15TNUM2PBS6F0, 0, 37, 6
A1ZSQ0ZRYGPK7D, 0, 33, 3
A1EHTFVSD0H7I6, 0, 42, 3
A27RK8VAH3TCX9, 0, 41, 16
ACL5MIC0HFI9J, 0, 37, 2
A3SXOJGQ8U0YA6, 0, 33, 9
A30J7WQV0ZNRXG, 0, 37, 15
A2CMKQ0U04GB78, 0, 35, 4
A240FRPD4MEXND, 0, 38, 24
A11V9QCUY0

# LOAD PREDICTIONS

In [121]:
predictions = pd.read_csv(f"{self_train_iterations[run_key]['prediction_path']}/{self_train_iterations[run_key]['dataset']}_split:test_seed:0_epoch:best_pred.csv")
len(predictions)

23962

In [137]:
predictions.sample(4)

,3
9272,1
10452,4
13898,3
15475,4


In [139]:
errors = 0
for ind, row in tqdm(datasets[-1].iterrows()):
    if self_train_iterations[run_key]['dataset'] == 'civilcomments':
        label = get_toxicity(row)
    elif self_train_iterations[run_key]['dataset'] == 'amazon':
        label = row['y']
        
    pred_row = predictions[predictions.index == row['id']]
    if len(pred_row) > 0:
        if label != pred_row['3'].values:
            errors += 1

print(f"ERRORS: {errors}")

150000it [01:02, 2384.68it/s]

ERRORS: 15488
